# Agent Comparison: LangChain vs DeepAgents

This notebook shows how to build the **same calendar agent** in two ways:

1. **LangChain** – `create_agent` with model, tools, and system prompt
2. **DeepAgents** – `create_deep_agent` with built-in planning, file system, and optional subagents

Both use the same tools and model; DeepAgents add planning and context management on top.

## 1. LangSmith setup

LangSmith gives you tracing, debugging, and visibility into agent runs.

**Setup steps:**

1. Sign up at [smith.langchain.com](https://smith.langchain.com)
2. Create an API key: **Settings → API Keys → Create API Key**
3. Add to your `.env` (see `.env.example`):
   ```
   LANGSMITH_API_KEY=your_api_key_here
   LANGSMITH_TRACING=true
   LANGSMITH_PROJECT=your-project-name
   ```

See `.env.example` in the project root. With this in place, runs are automatically sent to LangSmith.

In [4]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

# Optional: enable tracing if not set in .env
os.environ.setdefault("LANGSMITH_TRACING", "true")
os.environ.setdefault("LANGSMITH_PROJECT", "agent-comparison")

if os.getenv("LANGSMITH_API_KEY"):
    print("✅ LangSmith configured (traces will appear in your project)")
else:
    print("⚠️ LANGSMITH_API_KEY not set — add it to .env for tracing")

✅ LangSmith configured (traces will appear in your project)


## 2. Environment and shared pieces

Imports, model, and shared calendar tools used by both agents.

In [5]:
from typing import List, Dict
from langchain.chat_models import init_chat_model
from langchain.tools import tool

# Model (same for both agents)
model = init_chat_model("gpt-4o-mini", temperature=0)
print(f"✅ Model: {model.__class__.__name__}")

# Shared in-memory calendar
_calendar_events: List[Dict] = []

@tool
def read_calendar(date: str = None) -> str:
    """Read calendar events. If date is provided, filter events for that date (YYYY-MM-DD)."""
    if date:
        filtered = [e for e in _calendar_events if e.get("date") == date]
        if not filtered:
            return f"No events found for {date}"
        return "\n".join([f"- {e['title']} on {e['date']} at {e['time']} in {e.get('location', 'N/A')}" for e in filtered])
    if not _calendar_events:
        return "No events in calendar"
    return "\n".join([f"- {e['title']} on {e['date']} at {e['time']} in {e.get('location', 'N/A')}" for e in _calendar_events])

@tool
def write_calendar(title: str, date: str, time: str, location: str = "") -> str:
    """Create a new calendar event. Date: YYYY-MM-DD, time: HH:MM."""
    for event in _calendar_events:
        if event["date"] == date and event["time"] == time:
            return f"Conflict: '{event['title']}' already at {date} {time}"
    new_event = {"title": title, "date": date, "time": time, "location": location}
    _calendar_events.append(new_event)
    return f"Created '{title}' on {date} at {time} in {location}"

print("✅ Calendar tools defined")

✅ Model: ChatOpenAI
✅ Calendar tools defined


---

## 3. LangChain agent

Create an agent with **LangChain**: `create_agent(model, tools, system_prompt)`. Memory is optional (pass a checkpointer for conversation memory).

In [6]:
from langchain.agents import create_agent

SYSTEM_PROMPT = """You are a helpful calendar assistant. You can:
- Read events with read_calendar
- Create events with write_calendar
Use YYYY-MM-DD and HH:MM. Check for conflicts before creating. Current year: 2026."""

langchain_agent = create_agent(
    model=model,
    tools=[read_calendar, write_calendar],
    system_prompt=SYSTEM_PROMPT,
)

print("✅ LangChain agent created")

✅ LangChain agent created


In [7]:
# Run the LangChain agent
result = langchain_agent.invoke({
    "messages": [{"role": "user", "content": "Schedule a team standup on 2026-03-15 at 09:00 in Room A"}]
})
print("User:", "Schedule a team standup on 2026-03-15 at 09:00 in Room A")
print("Agent:", result["messages"][-1].content)

User: Schedule a team standup on 2026-03-15 at 09:00 in Room A
Agent: The team standup has been scheduled for 2026-03-15 at 09:00 in Room A.


---

## 4. DeepAgents

Create the **same** agent with **DeepAgents**: `create_deep_agent(...)`. You get the same tools plus:

- **write_todos** – plan multi-step tasks
- **File system** – `ls`, `read_file`, `write_file`, `edit_file` in agent state
- **task** – delegate to subagents (optional)

Pass a **checkpointer** (e.g. `MemorySaver()`) for conversation memory; both agents can run with or without it.

In [8]:
from deepagents import create_deep_agent
from langgraph.checkpoint.memory import MemorySaver

_calendar_events: List[Dict] = []

checkpointer = MemorySaver()

DEEP_SYSTEM_PROMPT = """You are a helpful calendar assistant with planning. You can:
- read_calendar, write_calendar (same as before)
- write_todos: break down complex tasks (built-in)
- File tools: ls, read_file, write_file, edit_file (built-in)
Use write_todos for multi-step requests. Current year: 2026."""

deep_agent = create_deep_agent(
    model=model,
    tools=[read_calendar, write_calendar],
    system_prompt=DEEP_SYSTEM_PROMPT,
    checkpointer=checkpointer,
)

print("✅ DeepAgent created (with write_todos + file system)")

✅ DeepAgent created (with write_todos + file system)


In [9]:
# Run the DeepAgent (use config with thread_id for memory)
config = {"configurable": {"thread_id": "comparison-thread-1"}}

result = deep_agent.invoke({
    "messages": [{"role": "user", "content": "Schedule three meetings next week: Mon 10:00, Wed 14:00, Fri 16:00. All in Room B. Then list what you scheduled."}]
}, config=config)

print("Agent:", result["messages"][-1].content)

Agent: I have scheduled the following meetings for next week in Room B:

- **Monday, October 2, 2026** at **10:00**
- **Wednesday, October 4, 2026** at **14:00**
- **Friday, October 6, 2026** at **16:00**


---

## 5. Best practices (DeepAgents)

**When to use subagents**

- **Context preservation** – Multi-step tasks (e.g. many web searches) fill the main agent’s context; subagents run in isolation and return only the final result.
- **Specialization** – Give a subagent domain-specific instructions and a focused tool set (e.g. only search).
- **Multi-model** – Subagents can use a different model (e.g. smaller/faster) than the main agent.
- **Parallelization** – Multiple subagents can run in parallel and hand results back to the main agent.

**Subagent design**

- **Descriptions**: Clear and specific so the main agent knows when to call which subagent.  
  ✅ "Conducts in-depth research using web search and synthesizes findings into a concise summary"  
  ❌ "Does research"
- **System prompts**: Include tool-usage guidance and output format (e.g. "Keep response under 500 words", "Cite sources").
- **Minimal tool sets**: Only give each subagent the tools it needs (e.g. research subagent gets search only, not calendar + search + email).

---

## 6. Subagents: research with Tavily

Subagents isolate work: the **research subagent** does multiple web searches in its own context and returns a single summary to the main agent. That keeps the main agent’s context small and avoids “context bloat.”

Here we use **Tavily** for search. For real search: set `TAVILY_API_KEY` in `.env` and install `tavily-python` (`pip install tavily-python`). Without the API key, a mock is used so the notebook still runs.

In [10]:
# Tavily search tool: real API if TAVILY_API_KEY is set, else mock
def _tavily_search_impl(query: str, max_results: int = 4) -> str:
    api_key = os.getenv("TAVILY_API_KEY")
    if api_key:
        try:
            from tavily import TavilyClient
            client = TavilyClient(api_key=api_key)
            response = client.search(query=query, max_results=max_results)
            results = response.get("results", [])
            if not results:
                return f"No results for: {query}"
            return "\n\n".join(
                f"[{i+1}] {r.get('title', '')}\n{r.get('content', '')}\nURL: {r.get('url', '')}"
                for i, r in enumerate(results)
            )
        except Exception as e:
            return f"Tavily search error: {e}. Using mock."
    # Mock for demo without API key
    return f'''Search results for "{query}":
1. Example Corp - Series B $50M (2025), CEO Jane Doe. Competitors: Acme Inc, Beta LLC.
2. Recent news: Example Corp announced partnership with TechCo (March 2025).
URLs: https://example.com/company, https://example.com/news'''

@tool
def tavily_search(query: str, max_results: int = 4) -> str:
    """Search the web for current information. Use for company research, events, or general lookup."""
    return _tavily_search_impl(query, max_results)

# Research subagent: focused tools + clear instructions
research_subagent = {
    "name": "research-specialist",
    "description": "Conducts in-depth research using web search and synthesizes findings into a concise summary. Use when you need company info, events, or multi-query research.",
    "system_prompt": """You are a thorough researcher. Your job is to:
1. Break down the research question into 2-4 focused search queries
2. Use tavily_search to gather information
3. Synthesize findings into a concise summary
4. Cite sources (title + URL) when making claims

Output format: Summary (2-3 short paragraphs), Key facts (bullets), Sources (with URLs). Keep response under 400 words.""",
    "tools": [tavily_search],
    "model": model,
}

# Supervisor agent with subagent
checkpointer2 = MemorySaver()
supervisor_prompt = """You are a coordinator. You have:
- read_calendar, write_calendar for scheduling
- task: delegate to subagents

Use the 'task' tool with name "research-specialist" when the user asks for company research, web search, or "find out about X". Use calendar tools for scheduling. Return a clear, consolidated answer."""

agent_with_subagent = create_deep_agent(
    model=model,
    tools=[read_calendar, write_calendar, tavily_search],
    system_prompt=supervisor_prompt,
    checkpointer=checkpointer2,
    subagents=[research_subagent],
)
print("✅ DeepAgent with research subagent created (Tavily: real if TAVILY_API_KEY set, else mock)")

✅ DeepAgent with research subagent created (Tavily: real if TAVILY_API_KEY set, else mock)


In [16]:
# Example: delegate company research to the subagent
config_super = {"configurable": {"thread_id": "subagent-demo-1"}}
result = agent_with_subagent.invoke({
    "messages": [{"role": "user", "content": "Research LangChain Inc: funding rounds, CEO, and main competitors. Give me a short summary with sources."}]
}, config=config_super)
print("User: Research LangChain Inc: funding rounds, CEO, and main competitors. Give me a short summary with sources.")
print("Agent:", result["messages"][-1].content)

User: Research LangChain Inc: funding rounds, CEO, and main competitors. Give me a short summary with sources.
Agent: Here's a summary of the research on LangChain Inc., focusing on its funding rounds, CEO, and main competitors:

### Summary
LangChain Inc. has emerged as a significant player in the AI infrastructure market, successfully raising a total of **$135 million** across three funding rounds. The most notable round was a **$125 million Series B**, which valued the company at **$1.1 billion**. This funding has been crucial for its growth and the development of tools for building AI agents and applications. The company is supported by prominent investors, including Sequoia Capital and Benchmark, indicating strong confidence in its business model and market potential ([Latenode](https://latenode.com/blog/ai-frameworks-technical-infrastructure/langchain-setup-tools-agents-memory/langchain-funding-valuation-2025-complete-financial-overview), [TexAu](https://www.texau.com/profiles/la

---

## 7. Skills: domain-specific web search (company intelligence)

**Skills** provide progressive disclosure: the agent sees skill **names and descriptions** first, and loads full instructions only when a skill is needed. That keeps the context window smaller.

This repo includes a **company intelligence** skill for domain-specific web search: how to query, what to look for, and how to format answers (funding, leadership, competitors, sources). Skill path: `.deepagents/skills/company-intelligence/SKILL.md`.

In [21]:
# Load skills from the project's .deepagents/skills directory
# Backend + skills API may vary by deepagents version; see docs for your version.
import os
_skills_path = os.path.abspath(".deepagents/skills")
agent_with_skills = None
if os.path.isdir(_skills_path):
    try:
        from deepagents.backends import FilesystemBackend
        _root = os.path.abspath(".")
        
        agent_with_skills = create_deep_agent(
            model=model,
            tools=[tavily_search],
            system_prompt="You are a research assistant. Use the company-intelligence skill when the user asks about companies to properly understand how to perform a company intelligence search.",
            checkpointer=MemorySaver(),
            backend=FilesystemBackend(root_dir=_root),
            skills=[_skills_path],
        )

        print("✅ DeepAgent with skills loaded from", _skills_path)
    except Exception as e:
        print("⚠️ Could not load skills (backend/skills API may differ):", e)
else:
    print("⚠️ .deepagents/skills not found; add SKILL.md files there to use skills")

✅ DeepAgent with skills loaded from /Users/mperini/Projects/workshop-planner-agent/.deepagents/skills


In [ ]:
# Optional: run the agent with company-intelligence skill (follows SKILL.md for query + output format)
if agent_with_skills:
    cfg = {"configurable": {"thread_id": "skills-demo-2"}}
    r = agent_with_skills.invoke({
        "messages": [{"role": "user", "content": "Research LangChain Inc: funding rounds, CEO, and main competitors."}]
    }, config=cfg)
    print("Agent (with company-intelligence skill):", r["messages"][-1].content[:800] + "..." if len(r["messages"][-1].content) > 800 else r["messages"][-1].content)

Agent (with company-intelligence skill): ### LangChain Inc Overview

- **Funding Rounds**: LangChain has raised a total of **$260 million** across **4 funding rounds**. The latest funding round was a **$125 million Series B** on **October 20, 2025**. Notable investors include **Sequoia Capital**, **IVP**, and **Sapphire Ventures**.

- **CEO**: The current CEO of LangChain is **Harrison Chase**.

- **Main Competitors**: LangChain ranks **2nd** among **538 active competitors** in the AI developer tools market. Some of its notable competitors include:
  - **OpenAI**
  - **Cohere**
  - **Hugging Face**
  - **Anthropic**
  - **Google DeepMind**

### Additional Information
LangChain is recognized for its open-source framework that facilitates the development of AI applications using large language models (LLMs). The company aims to pro...


---

## Summary

| | LangChain | DeepAgents |
|---|-----------|------------|
| **API** | `create_agent(model, tools, system_prompt)` | `create_deep_agent(...)` |
| **Planning** | Manual in prompt | `write_todos` built-in |
| **Context** | Conversation only | + file system in state |
| **Delegation** | Custom (e.g. supervisor) | `task` tool + subagents |
| **Procedures** | In system prompt | **Skills** (progressive disclosure) |

**Takeaways**

- **LangChain** agents are flexible: use for tool-calling, add memory (checkpointer), middleware, and custom patterns as needed.
- Use **DeepAgents** when you want planning, in-conversation files, subagents, and skills.
- **Subagents**: isolate context and specialize (e.g. research subagent with Tavily); give clear descriptions and minimal tool sets.
- **Skills**: store procedures in `.deepagents/skills/*/SKILL.md`; the agent sees descriptions first and loads full instructions when needed (e.g. company-intelligence for domain-specific web search).